<a href="https://colab.research.google.com/github/beryamosta/MLFloorPlan/blob/main/MLFloorPlan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print ('yazod')

yazod
